In [1]:
"""simulation for the Hyperdrive market"""
from __future__ import annotations
import json
import pandas as pd
import os
import numpy as np
import elfpy
from elfpy.wallet.wallet import Wallet, Long, Short
from elfpy.markets.hyperdrive import hyperdrive_assets
import elfpy.utils.apeworx_integrations as ape_utils
from collections import namedtuple
from elfpy import types
from elfpy.math import FixedPoint

In [2]:
# Hard coding location for now
trans_data = "hyperTransRecs_updated.json"

with open(trans_data, 'r', encoding='utf8') as f:
    json_data = json.load(f)

data = pd.DataFrame(json_data)

In [3]:
data.columns

Index(['blockHash', 'blockNumber', 'from', 'gas', 'gasPrice',
       'maxPriorityFeePerGas', 'maxFeePerGas', 'hash', 'input', 'nonce', 'to',
       'transactionIndex', 'value', 'type', 'accessList', 'chainId', 'v', 'r',
       's', 'timestamp', 'txreceipt_status', 'isError', 'contractAddress',
       'decoded_logs', 'decoded_input'],
      dtype='object')

In [4]:
timestamps = data['timestamp']
decoded_logs = data['decoded_logs'].str[0]

In [5]:
data = pd.concat([pd.json_normalize(decoded_logs), pd.json_normalize(data['decoded_input'])], axis=1)

In [6]:
data.columns

Index(['event', 'logIndex', 'transactionIndex', 'transactionHash', 'address',
       'blockHash', 'blockNumber', 'args.owner', 'args.spender', 'args.value',
       'args.operator', 'args.from', 'args.to', 'args.id',
       'block_info.shareReserves_', 'block_info.bondReserves_',
       'block_info.lpTotalSupply', 'block_info.sharePrice',
       'block_info.longsOutstanding_', 'block_info.longAverageMaturityTime_',
       'block_info.longBaseVolume_', 'block_info.shortsOutstanding_',
       'block_info.shortAverageMaturityTime_', 'block_info.shortBaseVolume_',
       'function_name', 'function_identifier', 'args._contribution',
       'args._apr', 'args._destination', 'args._asUnderlying', 'args._minApr',
       'args._maxApr', 'args._bondAmount', 'args._maxDeposit',
       'args._baseAmount', 'args._minOutput', 'args._shares',
       'args._maturityTime'],
      dtype='object')

In [7]:
data

,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,args.owner,args.spender,args.value,...,args._destination,args._asUnderlying,args._minApr,args._maxApr,args._bondAmount,args._maxDeposit,args._baseAmount,args._minOutput,args._shares,args._maturityTime
0,Approval,37.0,46.0,0x874c776966aab7ef153a4970647e357c5f364f4ecc8c...,0x11fE4B6AE13d2a6055C8D9cF65c55bac32B5d844,0x2175f5e1fa5992ae9ae66a97da8a61c2c49cc5ffc150...,8749440.0,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xF7F0de3744C82825D77EdA8ce78f07A916fB6bE7,1157920892373161954235709850086879078532699846...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TransferSingle,35.0,38.0,0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...,8749466.0,NaN,NaN,68414467676586586450000,...,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
3,TransferSingle,80.0,39.0,0xb664ed0f54ad9dd2612d011459a51118ef15ebb5e6a8...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xeadca78301ab734358d769df259865a9e25c09e45fce...,8813859.0,NaN,NaN,958385814236527142046,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
4,TransferSingle,14.0,3.0,0x178bcee69ddbf83ca5abe0e2af91759dfafc5d883f93...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xd5817e0d54d0e18a193b42daf43648f050679de223e1...,8816573.0,NaN,NaN,958373163550940235039,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,TransferSingle,10.0,7.0,0x97c9b926672acfc9bebcaa66e24cec1f9d7927ac2c7f...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x9e0f118b1ef5535d90fd215377a73fe245a08885fc2f...,9019605.0,NaN,NaN,558532369862596034560,...,0xFed2c446A218d26477b78f652B252a400F929436,True,NaN,NaN,NaN,NaN,NaN,0.0,558532369862596034560,NaN
661,TransferSingle,20.0,5.0,0xda982c5697703282227e36413b5039a949622d89dbde...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x37da42e49079e146efdaaa2666458792c7200c720b6e...,9019609.0,NaN,NaN,539151992294568427520,...,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,True,NaN,NaN,539151992294568427520,539151992294568427520,NaN,NaN,NaN,NaN
662,TransferSingle,125.0,17.0,0x7fea75be395d67638d5b05feaaf8c0b952d62dfde493...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x3066e0d71d5b04f9bd20a275133da9e5b1670cade29a...,9019612.0,NaN,NaN,619211525903964653563,...,0xFed2c446A218d26477b78f652B252a400F929436,True,NaN,NaN,NaN,NaN,609694378911564431360,0.0,NaN,NaN
663,TransferSingle,51.0,11.0,0x90bb8f9e0f9f4d313cf7ede9eefce757a94b2e776b25...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x13282f11a3cd9d2f9ed8650a8f3db95c2c73aa108cc9...,9019614.0,NaN,NaN,215712195810627633856,...,0x4612E8A93E2e089d074c25F41F3b11D853718f68,True,NaN,NaN,NaN,NaN,212411151625704079360,0.0,NaN,NaN


In [8]:
data = data[~data['args.id'].isna()]

In [9]:
data = data.reset_index(drop=True)

In [10]:
data

,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,args.owner,args.spender,args.value,...,args._destination,args._asUnderlying,args._minApr,args._maxApr,args._bondAmount,args._maxDeposit,args._baseAmount,args._minOutput,args._shares,args._maturityTime
0,TransferSingle,35.0,38.0,0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...,8749466.0,NaN,NaN,68414467676586586450000,...,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TransferSingle,80.0,39.0,0xb664ed0f54ad9dd2612d011459a51118ef15ebb5e6a8...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xeadca78301ab734358d769df259865a9e25c09e45fce...,8813859.0,NaN,NaN,958385814236527142046,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
2,TransferSingle,14.0,3.0,0x178bcee69ddbf83ca5abe0e2af91759dfafc5d883f93...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xd5817e0d54d0e18a193b42daf43648f050679de223e1...,8816573.0,NaN,NaN,958373163550940235039,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
3,TransferSingle,10.0,7.0,0xa8a6e994b9a42e64e68c75eee172abab529d196f00f0...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x58490923f5656b62b7737e557498382b7cf0fbb2961f...,8816603.0,NaN,NaN,958372980077611331087,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
4,TransferSingle,270.0,137.0,0x6f41f062b7b9c7c0ac1e7a29f137d7a005b6e9fa4215...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x432c4a9cecdaae17b1b07d16e8469043303d284cd667...,8816612.0,NaN,NaN,734520710813263931127,...,0xFed2c446A218d26477b78f652B252a400F929436,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,TransferSingle,22.0,6.0,0x82fd8ed9bfcdfd9ac13c20e430ad86905b0813ecab1a...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xd52fdfe95e893a445b484a2aeca7839a887667c5c75b...,9019603.0,NaN,NaN,944492011946648576584,...,0x841958527DFe4499fA234A1Acc247b29C90d1C21,True,0.0,100000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
593,TransferSingle,10.0,7.0,0x97c9b926672acfc9bebcaa66e24cec1f9d7927ac2c7f...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x9e0f118b1ef5535d90fd215377a73fe245a08885fc2f...,9019605.0,NaN,NaN,558532369862596034560,...,0xFed2c446A218d26477b78f652B252a400F929436,True,NaN,NaN,NaN,NaN,NaN,0.0,558532369862596034560,NaN
594,TransferSingle,20.0,5.0,0xda982c5697703282227e36413b5039a949622d89dbde...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x37da42e49079e146efdaaa2666458792c7200c720b6e...,9019609.0,NaN,NaN,539151992294568427520,...,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,True,NaN,NaN,539151992294568427520,539151992294568427520,NaN,NaN,NaN,NaN
595,TransferSingle,125.0,17.0,0x7fea75be395d67638d5b05feaaf8c0b952d62dfde493...,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x3066e0d71d5b04f9bd20a275133da9e5b1670cade29a...,9019612.0,NaN,NaN,619211525903964653563,...,0xFed2c446A218d26477b78f652B252a400F929436,True,NaN,NaN,NaN,NaN,609694378911564431360,0.0,NaN,NaN


In [11]:
data.iloc[0]

event                                                                      TransferSingle
logIndex                                                                             35.0
transactionIndex                                                                     38.0
transactionHash                         0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...
address                                        0xB311B825171AF5A60d69aAD590B857B1E5ed23a2
blockHash                               0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...
blockNumber                                                                     8749466.0
args.owner                                                                            NaN
args.spender                                                                          NaN
args.value                                                        68414467676586586450000
args.operator                                  0x13c9306e6569EfaBE095C7504b40d57A26432AeA
args.from 

In [12]:
data['args.id']

0                                                      0
1                                                      0
2                                                      0
3                                                      0
4                                                      0
                             ...                        
592                                                    0
593                                                    0
594    4523128485832663883733241601901871400518358776...
595                                           1715904000
596                                           1715904000
Name: args.id, Length: 597, dtype: object

In [13]:
prefix_mask = (1 << 248) - 1
prefix = data['args.id'].values >> 248

In [14]:
maturity_timestamp = prefix & prefix_mask

In [15]:
trade_type = pd.DataFrame(prefix).apply(lambda x: hyperdrive_assets.AssetIdPrefix(x.values).name, axis=1)

In [16]:
data['prefix'] = prefix
data['maturity_timestamp'] = maturity_timestamp
data['trade_type'] = trade_type

In [17]:
columns = [
    'event', 
    'address', 
    'transactionHash', 
    'blockNumber', 
    'blockHash', 
    'logIndex', 
    'transactionIndex', 
    'args.operator', 
    'args.from', 
    'args.to', 
    'args.id', 
    'args.value', 
    'prefix',
    'maturity_timestamp',
    'trade_type',
]

In [18]:
rename_columns = [
    'event_name',
    'contract_address',
    'transaction_hash',
    'block_number',
    'block_hash',
    'log_index',
    'transaction_index',
    'operator',
    'from',
    'to',
    'id',
    'value',
    'prefix',
    'maturity_timestamp',
    'trade_type',
]

In [19]:
renamed_data = data[columns]

In [20]:
renamed_data

,event,address,transactionHash,blockNumber,blockHash,logIndex,transactionIndex,args.operator,args.from,args.to,args.id,args.value,prefix,maturity_timestamp,trade_type
0,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...,8749466.0,0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...,35.0,38.0,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,0x0000000000000000000000000000000000000000,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,0,68414467676586586450000,0,0,LP
1,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xb664ed0f54ad9dd2612d011459a51118ef15ebb5e6a8...,8813859.0,0xeadca78301ab734358d769df259865a9e25c09e45fce...,80.0,39.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958385814236527142046,0,0,LP
2,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x178bcee69ddbf83ca5abe0e2af91759dfafc5d883f93...,8816573.0,0xd5817e0d54d0e18a193b42daf43648f050679de223e1...,14.0,3.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958373163550940235039,0,0,LP
3,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xa8a6e994b9a42e64e68c75eee172abab529d196f00f0...,8816603.0,0x58490923f5656b62b7737e557498382b7cf0fbb2961f...,10.0,7.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958372980077611331087,0,0,LP
4,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x6f41f062b7b9c7c0ac1e7a29f137d7a005b6e9fa4215...,8816612.0,0x432c4a9cecdaae17b1b07d16e8469043303d284cd667...,270.0,137.0,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0xFed2c446A218d26477b78f652B252a400F929436,0,734520710813263931127,0,0,LP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x82fd8ed9bfcdfd9ac13c20e430ad86905b0813ecab1a...,9019603.0,0xd52fdfe95e893a445b484a2aeca7839a887667c5c75b...,22.0,6.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,944492011946648576584,0,0,LP
593,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x97c9b926672acfc9bebcaa66e24cec1f9d7927ac2c7f...,9019605.0,0x9e0f118b1ef5535d90fd215377a73fe245a08885fc2f...,10.0,7.0,0xFed2c446A218d26477b78f652B252a400F929436,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0,558532369862596034560,0,0,LP
594,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xda982c5697703282227e36413b5039a949622d89dbde...,9019609.0,0x37da42e49079e146efdaaa2666458792c7200c720b6e...,20.0,5.0,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,0x0000000000000000000000000000000000000000,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,4523128485832663883733241601901871400518358776...,539151992294568427520,1,1,LONG
595,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x7fea75be395d67638d5b05feaaf8c0b952d62dfde493...,9019612.0,0x3066e0d71d5b04f9bd20a275133da9e5b1670cade29a...,125.0,17.0,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0xFed2c446A218d26477b78f652B252a400F929436,1715904000,619211525903964653563,0,0,LP


In [21]:
renamed_data.columns = rename_columns

In [22]:
renamed_data

,event_name,contract_address,transaction_hash,block_number,block_hash,log_index,transaction_index,operator,from,to,id,value,prefix,maturity_timestamp,trade_type
0,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...,8749466.0,0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...,35.0,38.0,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,0x0000000000000000000000000000000000000000,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,0,68414467676586586450000,0,0,LP
1,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xb664ed0f54ad9dd2612d011459a51118ef15ebb5e6a8...,8813859.0,0xeadca78301ab734358d769df259865a9e25c09e45fce...,80.0,39.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958385814236527142046,0,0,LP
2,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x178bcee69ddbf83ca5abe0e2af91759dfafc5d883f93...,8816573.0,0xd5817e0d54d0e18a193b42daf43648f050679de223e1...,14.0,3.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958373163550940235039,0,0,LP
3,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xa8a6e994b9a42e64e68c75eee172abab529d196f00f0...,8816603.0,0x58490923f5656b62b7737e557498382b7cf0fbb2961f...,10.0,7.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,958372980077611331087,0,0,LP
4,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x6f41f062b7b9c7c0ac1e7a29f137d7a005b6e9fa4215...,8816612.0,0x432c4a9cecdaae17b1b07d16e8469043303d284cd667...,270.0,137.0,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0xFed2c446A218d26477b78f652B252a400F929436,0,734520710813263931127,0,0,LP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x82fd8ed9bfcdfd9ac13c20e430ad86905b0813ecab1a...,9019603.0,0xd52fdfe95e893a445b484a2aeca7839a887667c5c75b...,22.0,6.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0,944492011946648576584,0,0,LP
593,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x97c9b926672acfc9bebcaa66e24cec1f9d7927ac2c7f...,9019605.0,0x9e0f118b1ef5535d90fd215377a73fe245a08885fc2f...,10.0,7.0,0xFed2c446A218d26477b78f652B252a400F929436,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0,558532369862596034560,0,0,LP
594,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xda982c5697703282227e36413b5039a949622d89dbde...,9019609.0,0x37da42e49079e146efdaaa2666458792c7200c720b6e...,20.0,5.0,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,0x0000000000000000000000000000000000000000,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,4523128485832663883733241601901871400518358776...,539151992294568427520,1,1,LONG
595,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x7fea75be395d67638d5b05feaaf8c0b952d62dfde493...,9019612.0,0x3066e0d71d5b04f9bd20a275133da9e5b1670cade29a...,125.0,17.0,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0xFed2c446A218d26477b78f652B252a400F929436,1715904000,619211525903964653563,0,0,LP


In [23]:
#block_info_data
data.columns

Index(['event', 'logIndex', 'transactionIndex', 'transactionHash', 'address',
       'blockHash', 'blockNumber', 'args.owner', 'args.spender', 'args.value',
       'args.operator', 'args.from', 'args.to', 'args.id',
       'block_info.shareReserves_', 'block_info.bondReserves_',
       'block_info.lpTotalSupply', 'block_info.sharePrice',
       'block_info.longsOutstanding_', 'block_info.longAverageMaturityTime_',
       'block_info.longBaseVolume_', 'block_info.shortsOutstanding_',
       'block_info.shortAverageMaturityTime_', 'block_info.shortBaseVolume_',
       'function_name', 'function_identifier', 'args._contribution',
       'args._apr', 'args._destination', 'args._asUnderlying', 'args._minApr',
       'args._maxApr', 'args._bondAmount', 'args._maxDeposit',
       'args._baseAmount', 'args._minOutput', 'args._shares',
       'args._maturityTime', 'prefix', 'maturity_timestamp', 'trade_type'],
      dtype='object')

In [24]:
block_info_data = data[[c for c in data.columns if 'block_info.' in c]]
block_info_data

,block_info.shareReserves_,block_info.bondReserves_,block_info.lpTotalSupply,block_info.sharePrice,block_info.longsOutstanding_,block_info.longAverageMaturityTime_,block_info.longBaseVolume_,block_info.shortsOutstanding_,block_info.shortAverageMaturityTime_,block_info.shortBaseVolume_
0,50000000000000000000000,86828935353173172900000,68414467676586586450000,1.000000e+18,0,0.000000e+00,0,0,0.000000e+00,0
1,47831784082305287728729,91048383734353921793143,69372853490823113592046,1.000309e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
2,48532201249594206792567,92381636345456968430464,70331226654374053827085,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
3,49232618284284535862805,93714888704156983568950,71289599634451665158172,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
4,49769435190747076198331,94736726225638254619806,72024120345264929089299,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
...,...,...,...,...,...,...,...,...,...,...
592,129727425606990786688762,336632614765094409056057,184664102568896043062018,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
593,129530671106860213437793,336122052086957175227957,184105570199033447027458,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
594,129002349108095444039167,336660325504945911706246,184105570199033447027458,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,90065482934054616268800,1.713596e+09,88686564219203450434515
595,129611146266371572769710,336042021905585788369888,184105570199033447027458,1.001300e+18,65378994331869867879006,1.713567e+09,64566003380758624735819,90065482934054616268800,1.713596e+09,88686564219203450434515


In [25]:
block_info_data.columns = [
    "share_reserves",
    "bond_reserves",
    "lp_total_supply",
    "share_price",
    "longs_outstanding",
    "average_maturity_time",
    "long_base_volume",
    "shorts_outstanding",
    "short_average_maturity_time",
    "short_base_volume",
]

In [26]:
block_info_data

,share_reserves,bond_reserves,lp_total_supply,share_price,longs_outstanding,average_maturity_time,long_base_volume,shorts_outstanding,short_average_maturity_time,short_base_volume
0,50000000000000000000000,86828935353173172900000,68414467676586586450000,1.000000e+18,0,0.000000e+00,0,0,0.000000e+00,0
1,47831784082305287728729,91048383734353921793143,69372853490823113592046,1.000309e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
2,48532201249594206792567,92381636345456968430464,70331226654374053827085,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
3,49232618284284535862805,93714888704156983568950,71289599634451665158172,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
4,49769435190747076198331,94736726225638254619806,72024120345264929089299,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
...,...,...,...,...,...,...,...,...,...,...
592,129727425606990786688762,336632614765094409056057,184664102568896043062018,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
593,129530671106860213437793,336122052086957175227957,184105570199033447027458,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
594,129002349108095444039167,336660325504945911706246,184105570199033447027458,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,90065482934054616268800,1.713596e+09,88686564219203450434515
595,129611146266371572769710,336042021905585788369888,184105570199033447027458,1.001300e+18,65378994331869867879006,1.713567e+09,64566003380758624735819,90065482934054616268800,1.713596e+09,88686564219203450434515


In [27]:
trade_data = pd.concat([renamed_data, block_info_data], axis=1)
trade_data

,event_name,contract_address,transaction_hash,block_number,block_hash,log_index,transaction_index,operator,from,to,...,share_reserves,bond_reserves,lp_total_supply,share_price,longs_outstanding,average_maturity_time,long_base_volume,shorts_outstanding,short_average_maturity_time,short_base_volume
0,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x752915e8e7ce11ddbb0cc21aea6c36eda213a810c00b...,8749466.0,0x2f260b8580eb4e3091c5c83a9027300aa8901f1638d2...,35.0,38.0,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,0x0000000000000000000000000000000000000000,0x13c9306e6569EfaBE095C7504b40d57A26432AeA,...,50000000000000000000000,86828935353173172900000,68414467676586586450000,1.000000e+18,0,0.000000e+00,0,0,0.000000e+00,0
1,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xb664ed0f54ad9dd2612d011459a51118ef15ebb5e6a8...,8813859.0,0xeadca78301ab734358d769df259865a9e25c09e45fce...,80.0,39.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,...,47831784082305287728729,91048383734353921793143,69372853490823113592046,1.000309e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
2,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x178bcee69ddbf83ca5abe0e2af91759dfafc5d883f93...,8816573.0,0xd5817e0d54d0e18a193b42daf43648f050679de223e1...,14.0,3.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,...,48532201249594206792567,92381636345456968430464,70331226654374053827085,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
3,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xa8a6e994b9a42e64e68c75eee172abab529d196f00f0...,8816603.0,0x58490923f5656b62b7737e557498382b7cf0fbb2961f...,10.0,7.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,...,49232618284284535862805,93714888704156983568950,71289599634451665158172,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
4,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x6f41f062b7b9c7c0ac1e7a29f137d7a005b6e9fa4215...,8816612.0,0x432c4a9cecdaae17b1b07d16e8469043303d284cd667...,270.0,137.0,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,0xFed2c446A218d26477b78f652B252a400F929436,...,49769435190747076198331,94736726225638254619806,72024120345264929089299,1.000323e+18,0,0.000000e+00,0,2903750000000000000000,1.712700e+09,2869443834473984319634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x82fd8ed9bfcdfd9ac13c20e430ad86905b0813ecab1a...,9019603.0,0xd52fdfe95e893a445b484a2aeca7839a887667c5c75b...,22.0,6.0,0x841958527DFe4499fA234A1Acc247b29C90d1C21,0x0000000000000000000000000000000000000000,0x841958527DFe4499fA234A1Acc247b29C90d1C21,...,129727425606990786688762,336632614765094409056057,184664102568896043062018,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
593,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0x97c9b926672acfc9bebcaa66e24cec1f9d7927ac2c7f...,9019605.0,0x9e0f118b1ef5535d90fd215377a73fe245a08885fc2f...,10.0,7.0,0xFed2c446A218d26477b78f652B252a400F929436,0xFed2c446A218d26477b78f652B252a400F929436,0x0000000000000000000000000000000000000000,...,129530671106860213437793,336122052086957175227957,184105570199033447027458,1.001300e+18,64759782805965903225443,1.713544e+09,63956419272532952779864,89526330941760047841280,1.713582e+09,88157562584667847430197
594,TransferSingle,0xB311B825171AF5A60d69aAD590B857B1E5ed23a2,0xda982c5697703282227e36413b5039a949622d89dbde...,9019609.0,0x37da42e49079e146efdaaa2666458792c7200c720b6e...,20.0,5.0,0x2C47e2A9948d10aD873109eB85c6F2e45186277b,0x000000000000

In [28]:
def get_wallet_from_onchain_trade_info(
    address: str,
    trades: pd.DataFrame,
    index: int = 0,
    add_to_existing_wallet: Wallet | None = None,
) -> Wallet:
    

    # pylint: disable=too-many-arguments, too-many-branches

    r"""Construct wallet balances from on-chain trade info.

    Arguments
    ---------
    address : str
        Address of the wallet.
    info : OnChainTradeInfo
        On-chain trade info.
    index : int
        Index of the wallet among ALL agents.

    Returns
    -------
    Wallet
        Wallet with Short, Long, and LP positions.
    """
    share_price = trades.share_price
    
    # TODO: remove restriction forcing Wallet index to be an int (issue #415)
    if add_to_existing_wallet is None:
        wallet = Wallet(
            address=index,
            balance=types.Quantity(
                amount=FixedPoint(scaled_value=0), unit=types.TokenType.BASE
            ),
        )
    else:
        wallet = add_to_existing_wallet
    
    position_id = trades["id"]
    trades_in_position = ((trades["from"] == address) | (trades["to"] == address)) & (
        trades["id"] == position_id
    )
    
    positive_balance = int(trades.loc[(trades_in_position) & (trades["to"] == address), "value"].sum())
    negative_balance = int(trades.loc[(trades_in_position) & (trades["from"] == address), "value"].sum())
    balance = positive_balance - negative_balance
    
    asset_prefix = trades["prefix"].iloc[0]
    asset_type = trades["trade_type"].iloc[0]
    maturity = trades["maturity_timestamp"].iloc[0]
    mint_time = maturity - elfpy.SECONDS_IN_YEAR

    # check if there's an outstanding balance
    if balance != 0:
        if asset_type == "SHORT":
            # loop across all the positions owned by this wallet
            sum_product_of_open_share_price_and_value, sum_value = 0, 0
            
            # WEIGHTED AVERAGE ACROSS A BUNCH OF TRADES
            for specific_trade in trades_in_position.index[trades_in_position]:
                value = trades.loc[specific_trade, "value"]
                value *= -1 if trades.loc[specific_trade, "from"] == address else 1
                sum_value += value
                sum_product_of_open_share_price_and_value += (
                    value * share_price[trades.loc[specific_trade, "block_number"]]
                )
            open_share_price = int(sum_product_of_open_share_price_and_value / sum_value)

            # WEIGHTED AVERAGR FROM A MARGINAL UPDATE
            previous_balance = wallet.shorts[mint_time].balance if mint_time in wallet.shorts else 0
            previous_share_price = wallet.shorts[mint_time].open_share_price if mint_time in wallet.shorts else 0

            marginal_position_change = FixedPoint(scaled_value=balance)
            marginal_open_share_price = FixedPoint(scaled_value=trades.open_share_price)

            new_balance = previous_balance + marginal_position_change

            if new_balance == 0:
                # remove key of "mint_time" from the "wallet.shorts" dict
                wallet.shorts.pop(mint_time, None)
            else:
                new_open_share_price = previous_balance * previous_share_price \
                    + marginal_position_change * marginal_open_share_price \
                    / ( previous_balance + marginal_position_change )
                wallet.shorts.update(
                    {
                        mint_time: Short(
                            balance=new_balance,
                            open_share_price=new_open_share_price,
                        )
                    }
                )
        elif asset_type == "LONG":
            previous_balance = wallet.longs[mint_time].balance if mint_time in wallet.longs else 0
            new_balance = previous_balance + FixedPoint(scaled_value=balance)
            if new_balance == 0:
                # remove key of "mint_time" from the "wallet.longs" dict
                wallet.longs.pop(mint_time, None)
            wallet.longs.update({mint_time: Long(balance=new_balance)})
        elif asset_type == "LP":
            wallet.lp_tokens += FixedPoint(scaled_value=balance)
    return wallet

In [29]:
trade_data.columns

Index(['event_name', 'contract_address', 'transaction_hash', 'block_number',
       'block_hash', 'log_index', 'transaction_index', 'operator', 'from',
       'to', 'id', 'value', 'prefix', 'maturity_timestamp', 'trade_type',
       'share_reserves', 'bond_reserves', 'lp_total_supply', 'share_price',
       'longs_outstanding', 'average_maturity_time', 'long_base_volume',
       'shorts_outstanding', 'short_average_maturity_time',
       'short_base_volume'],
      dtype='object')

In [30]:
agents = trade_data['operator'].value_counts().index.tolist()

In [31]:
# agent_trades = {a: trade_data[trade_data['operator'] == a] for a in agents}
agent_wallets = {
    a: Wallet(
        address=agents.index(a),
        balance=types.Quantity(amount=FixedPoint(0), unit=types.TokenType.BASE),
    )
    for a in agents
    }

for a,v in agent_wallets.items():
    print(v)

Wallet(address=0, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), longs={}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0"))
Wallet(address=1, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), longs={}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0"))
Wallet(address=2, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), longs={}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0"))
Wallet(address=3, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), longs={}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0"))
Wallet(address=4, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), lo

In [ ]:
def get_wallet_state(agent_wallet: Wallet, market: HyperdriveMarket) -> dict[str, FixedPoint]:
    r"""The wallet's current state of public variables

    .. todo:: This will go away once we finish refactoring the state
    """
    lp_token_value = FixedPoint(0)
    # proceed further only if the agent has LP tokens and avoid divide by zero
    if agent_wallet.lp_tokens > FixedPoint(0) and market.market_state.lp_total_supply > FixedPoint(0):
        share_of_pool = agent_wallet.lp_tokens / market.market_state.lp_total_supply
        pool_value = (
            market.market_state.bond_reserves * market.spot_price  # in base
            + market.market_state.share_reserves * market.market_state.share_price  # in base
        )
        lp_token_value = pool_value * share_of_pool  # in base
    share_reserves = market.market_state.share_reserves
    # compute long values in units of base
    longs_value = FixedPoint(0)
    longs_value_no_mock = FixedPoint(0)
    for mint_time, long in agent_wallet.longs.items():
        if long.balance > FixedPoint(0) and share_reserves:
            balance = hyperdrive_actions.calc_close_long(
                bond_amount=long.balance,
                market_state=market.market_state,
                position_duration=market.position_duration,
                pricing_model=market.pricing_model,
                block_time=market.block_time.time,
                mint_time=mint_time,
                is_trade=True,
            )[1].balance.amount
        else:
            balance = FixedPoint(0)
        longs_value += balance
        longs_value_no_mock += long.balance * market.spot_price
    # compute short values in units of base
    shorts_value = FixedPoint(0)
    shorts_value_no_mock = FixedPoint(0)
    for mint_time, short in agent_wallet.shorts.items():
        balance = FixedPoint(0)
        if (
            short.balance > FixedPoint(0)
            and share_reserves > FixedPoint(0)
            and market.market_state.bond_reserves - market.market_state.bond_buffer > short.balance
        ):
            balance = hyperdrive_actions.calc_close_short(
                bond_amount=short.balance,
                market_state=market.market_state,
                position_duration=market.position_duration,
                pricing_model=market.pricing_model,
                block_time=market.block_time.time,
                mint_time=mint_time,
                open_share_price=short.open_share_price,
            )[1].balance.amount
        shorts_value += balance
        base_no_mock = short.balance * (FixedPoint("1.0") - market.spot_price)
        shorts_value_no_mock += base_nomock
    return {
        f"agent{agent_wallet.address}_base": agentwallet.balance.amount,
        f"agent{agent_wallet.address}_lp_tokens": lp_tokenvalue,
        f"agent{agent_wallet.address}_num_longs": FixedPoint(len(agentwallet.longs)),
        f"agent{agent_wallet.address}_num_shorts": FixedPoint(len(agentwallet.shorts)),
        f"agent{agent_wallet.address}_total_longs": longsvalue,
        f"agent{agent_wallet.address}_total_shorts": shortsvalue,
        f"agent{agent_wallet.address}_total_longs_no_mock": longs_value_nomock,
        f"agent{agent_wallet.address}_total_shorts_no_mock": shorts_value_no_mock,
    }

In [32]:
# for index, (address, trades) in enumerate(agent_trades.items()):
for idx,row in trade_data.iterrows():
    agent = row.operator
    # get their wallet
    wallet = agent_wallets[agent]

    marginal_trades = pd.DataFrame(row).T

    # pass in one trade at a time, and get out the updated wallet
    agent_wallets[agent] = get_wallet_from_onchain_trade_info(address=agent, trades = marginal_trades, index=agents.index(agent), add_to_existing_wallet=agent_wallets[agent],)
    
    pnl = 
    
    

In [33]:
agent_wallets

{'0x841958527DFe4499fA234A1Acc247b29C90d1C21': Wallet(address=0, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("81483.82629908315746228"), longs={FixedPoint("-31535999.000000001644167168"): Long(balance=FixedPoint("21120.825906708255080448"))}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0")),
 '0xFed2c446A218d26477b78f652B252a400F929436': Wallet(address=1, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("49744.392554664852014075"), longs={FixedPoint("-31535999.000000001644167168"): Long(balance=FixedPoint("30585.50393016945836032"))}, shorts={}, withdraw_shares=FixedPoint("0.0"), borrows={}, fees_paid=FixedPoint("0.0")),
 '0x2C47e2A9948d10aD873109eB85c6F2e45186277b': Wallet(address=2, balance=Quantity(amount=FixedPoint("0.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("27710.379380450123331806"), longs={FixedPoint("-31535999.000000001644167